In [1]:
%pip install spacy
import spacy
!python3.11 -m spacy download en_core_web_sm

%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 16.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
import numpy as np
import spacy
from spacy.tokens import Doc
from collections import defaultdict
from typing import List, Tuple

import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/shaojh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/shaojh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
#test

with open('translations/wolof.json', 'r') as fp:
    wolof = json.load(fp)
print(wolof['passages']['1']['official'])
print(wolof['passages']['1']['roundtrip_chatgpt'])
print(wolof['passages']['1']['roundtrip_google'])

Ca njàlbéen ga Yàlla sàkk na asamaan ak suuf. Suuf nag daa maase woon te wéet, lëndëm muur ndox mu xóot mi, lale ca kaw, Noowug Yàlla di wër, tiim ndox mi.Ba loolu amee Yàlla ne: «Na leer nekk,» leer daldi am. Yàlla gis ne leer gi lu baax la, mu xàjjale leer ak lëndëm. Yàlla tudde leer gi bëccëg, lëndëm gi guddi. Ngoon jot, suba dugg, muy bés ba jëkk.
In the beginning, God created the heavens and the earth. The earth had no form and was empty; and darkness was over the surface of the deep. And the Spirit of God was hovering over the waters. And God said, 'Let there be light.' And there was light. And God saw that the light was good; and He separated the light from the darkness. God called the light 'Day,' and He called the darkness 'Night.' And there was evening and there was morning—the first day.
In the beginning God created the heavens and the earth. And the earth was without form, and void; Darkness covered the deep. And the Spirit of God moved upon the face of the waters. And God 

In [4]:
nlp = spacy.load("en_core_web_sm")

# parse text into syntax tree
def extract_dependencies(text: str) -> List[Tuple[str, str, str]]:
    doc = nlp(text)
    deps = []
    for sent in doc.sents:
        for token in sent:
            deps.append((token.text, token.dep_, token.head.text))
    return deps

# compare two dependency trees by counting matching dependency triples
def compare_trees(deps1, deps2):
    matches = 0
    for word1, dep1, head1 in deps1:
        for word2, dep2, head2 in deps2:
            if dep1 == dep2 and are_synonyms(word1, word2) and are_synonyms(head1, head2):
                matches += 1
                break
    return matches / max(len(deps1), len(deps2)) if deps1 and deps2 else 0

# detects if two words are synonyms
def are_synonyms(word1, word2):
    similarity_threshold = 0.75

    if word1.lower() == word2.lower():
        return True
    
    syns1 = wn.synsets(word1)
    syns2 = wn.synsets(word2)
    
    for s1 in syns1:
        for s2 in syns2:
            if s1.wup_similarity(s2) and s1.wup_similarity(s2) > similarity_threshold:
                return True
    return False

In [ ]:
language_strings = ["malay", "nahuatl", "quechua",  "wolof", "tibetan", "spanish"] #Tibetan removed due to formatting errors
language_jsons = {}
for language_string in language_strings:
    language_json_string = "translations/" + language_string + ".json"

    with open(language_json_string, 'r', encoding='utf-8') as fp:
        language_json = json.load(fp)
        language_jsons[language_string] = language_json

chatgpt_scores = {}
google_scores = {}
        
# print(language_jsons)

for language_string, language_json in language_jsons.items():

    # print("\n" + language_string + "\n")
    chatgpt_scores[language_string] = []
    google_scores[language_string] = []

    for passage_number in language_json['passages']:

        passage = language_json['passages'][passage_number]

        # print(passage)

        english = passage['english']
        roundtrip_chatgpt = passage['roundtrip_chatgpt']
        roundtrip_google = passage['roundtrip_google']
        
        # print(english)
        # print(roundtrip_chatgpt)
        # print(roundtrip_google)

        # extract dependency trees
        english_deps = extract_dependencies(english)
        chatgpt_deps = extract_dependencies(roundtrip_chatgpt)
        google_deps = extract_dependencies(roundtrip_google)

        # compare trees
        score_chatgpt = compare_trees(english_deps, chatgpt_deps)
        score_google = compare_trees(english_deps, google_deps)

        chatgpt_scores[language_string].append(score_chatgpt)
        google_scores[language_string].append(score_google)

        # print the comparison
        print("Similarity with ChatGPT roundtrip:", score_chatgpt)
        print("Similarity with Google roundtrip:", score_google)

        # print out the trees
        # print("\nChatGPT dependencies:")
        # for dep in chatgpt_deps[:10]:  # limit output
        #     print(dep)

        # print("\nGoogle dependencies:")
        # for dep in google_deps[:10]:
        #     print(dep)


Similarity with ChatGPT roundtrip: 0.8349514563106796
Similarity with Google roundtrip: 0.9320388349514563
Similarity with ChatGPT roundtrip: 0.5703703703703704
Similarity with Google roundtrip: 0.5925925925925926
Similarity with ChatGPT roundtrip: 0.5660377358490566
Similarity with Google roundtrip: 1.0
Similarity with ChatGPT roundtrip: 0.6233766233766234
Similarity with Google roundtrip: 0.8142857142857143
Similarity with ChatGPT roundtrip: 0.6875
Similarity with Google roundtrip: 0.6666666666666666
Similarity with ChatGPT roundtrip: 0.6850828729281768
Similarity with Google roundtrip: 0.8066298342541437
Similarity with ChatGPT roundtrip: 0.7037037037037037
Similarity with Google roundtrip: 0.970873786407767
Similarity with ChatGPT roundtrip: 0.1476510067114094
Similarity with Google roundtrip: 0.5555555555555556
Similarity with ChatGPT roundtrip: 0.07246376811594203
Similarity with Google roundtrip: 0.8225806451612904
Similarity with ChatGPT roundtrip: 0.06666666666666667
Similarit

TypeError: can only concatenate str (not "numpy.float64") to str

In [8]:

for language_string in language_strings:

    
    
    print(language_string + " Statistics")

    curr_chatgpt_scores = np.array(chatgpt_scores[language_string])
    curr_google_scores = np.array(google_scores[language_string])
    # print(all_values)

    chatgpt_score_mean = np.mean(curr_chatgpt_scores)
    chatgpt_score_median = np.median(curr_chatgpt_scores)
    google_score_mean = np.mean(curr_google_scores)
    google_score_median = np.median(curr_google_scores)

    # Score Difference Statistics

    score_differences = curr_chatgpt_scores - curr_google_scores

    score_differences_mean = np.mean(np.abs(score_differences))
    score_differences_median = np.median(np.abs(score_differences))

    chatgpt_better = np.sum(score_differences > 0)
    chatgpt_better_percentage = chatgpt_better / len(score_differences)
    google_better = np.sum(score_differences < 0)
    google_better_percentage = google_better / len(score_differences)

    print("ChatGPT Score Mean: ", chatgpt_score_mean)
    print("ChatGPT Score Median: ", chatgpt_score_median)
    print("Google Score Mean: ", google_score_mean)
    print("Google Score Median: ", google_score_median)

    print("Score Differences: ChatGPT Scores - Google Scores")
    print("Score Differences Mean: ", score_differences_mean)
    print("Score Differences Median: ", score_differences_median)
    print("Percentage where ChatGPT Performed Better: ", chatgpt_better_percentage)
    print("Percentage where Google Performed Better: ", google_better_percentage)

    print("\n")

# cumulative statistics

print("Cumulative Statistics")

all_chatgpt_scores = np.array([item for sublist in chatgpt_scores.values() for item in sublist])
all_google_scores = np.array([item for sublist in google_scores.values() for item in sublist])
# print(all_values)

chatgpt_score_mean = np.mean(all_chatgpt_scores)
chatgpt_score_median = np.median(all_chatgpt_scores)
google_score_mean = np.mean(all_google_scores)
google_score_median = np.median(all_google_scores)


score_differences = all_chatgpt_scores - all_google_scores

score_differences_mean = np.mean(np.abs(score_differences))
score_differences_median = np.median(np.abs(score_differences))

chatgpt_better = np.sum(score_differences > 0)
chatgpt_better_percentage = chatgpt_better / len(score_differences)
google_better = np.sum(score_differences < 0)
google_better_percentage = google_better / len(score_differences)

print("Cumulative ChatGPT Score Mean: ", chatgpt_score_mean)
print("Cumulative ChatGPT Score Median: ", chatgpt_score_median)
print("Cumulative Google Score Mean: ", google_score_mean)
print("Cumulative Google Score Median: ", google_score_median)

print("Cumulative Score Differences: ChatGPT Scores - Google Scores")
print("Cumulative Score Differences Mean: ", score_differences_mean)
print("Cumulative Score Differences Median: ", score_differences_median)
print("Cumulative Percentage where ChatGPT Performed Better: ", chatgpt_better_percentage)
print("Cumulative Percentage where Google Performed Better: ", google_better_percentage)

# for (c_score, g_score) in zip(chatgpt_scores, google_scores):


malay Statistics
ChatGPT Score Mean:  0.6612198431391512
ChatGPT Score Median:  0.6542297481524001
Google Score Mean:  0.8020356071250957
Google Score Median:  0.810457774269929
Score Differences: ChatGPT Scores - Google Scores
Score Differences Mean:  0.14776020843038892
Score Differences Median:  0.10931716998337182
Percentage where ChatGPT Performed Better:  0.16666666666666666
Percentage where Google Performed Better:  0.8333333333333334


nahuatl Statistics
ChatGPT Score Mean:  0.2113303426095368
ChatGPT Score Median:  0.11005738741367571
Google Score Mean:  0.7091870689465046
Google Score Median:  0.7784530386740331
Score Differences: ChatGPT Scores - Google Scores
Score Differences Mean:  0.497856726336968
Score Differences Median:  0.4785405787965893
Percentage where ChatGPT Performed Better:  0.0
Percentage where Google Performed Better:  1.0


quechua Statistics
ChatGPT Score Mean:  0.2839254178315912
ChatGPT Score Median:  0.22981898238747553
Google Score Mean:  0.7911034735